*이탤릭체 텍스트*# [FOMC 일정과 주가 변동성 분석] 프로젝트 흐름

  0. 목표 정의
미국 연준의 FOMC 회의 일정과 그 결과(금리 인상/동결/인하)가 미국 증시(S&P 500 등)에 단기적으로 어떤 영향을 주는지 분석
→ 주가 수익률 및 변동성(Volatility)이 발표일 전후로 어떻게 달라지는지를 시계열적으로 분석

1. 데이터 수집

1.1 FOMC 일정 및 정책 발표 수집
데이터 항목: 날짜, 기준금리 결정, 금리 변화 여부 (인상/동결/인하), 성명서 요약(optional)

수집 방법: 연준 공식 홈페이지 (FOMC calendar)

참고로 수동 수집도 가능 (10년치 일정 많지 않음)

2.1 주가 데이터 수집
대상:

S&P 500 지수 (^GSPC)

NASDAQ (^IXIC)

VIX 변동성 지수 (^VIX)

도구: yfinance (간단하고 안정적)

In [1]:
import yfinance as yf

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# 1. 데이터 수집
'''
야후 파이낸스 api를 이용 S&P 500의 데이터를 약10년치를 수집
'''
sp500 = yf.download("^GSPC", start="2015-01-01", end="2025-04-01")
sp500.reset_index(inplace=True)
sp500.head()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


Price,Date,Close,High,Low,Open,Volume
Ticker,,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
0,2015-01-02,2058.199951,2072.360107,2046.040039,2058.899902,2708700000
1,2015-01-05,2020.579956,2054.439941,2017.339966,2054.439941,3799120000
2,2015-01-06,2002.609985,2030.250000,1992.439941,2022.150024,4460110000
3,2015-01-07,2025.900024,2029.609985,2005.550049,2005.550049,3805480000
4,2015-01-08,2062.139893,2064.080078,2030.609985,2030.609985,3934010000


In [4]:
# 2. 데이터 전처리
import pandas as pd
import numpy as np

In [8]:
# FOMC 일정을 데이터프레임화

import pandas as pd
import re
from datetime import datetime

# FOMC 일정 데이터 준비
fomc_data = []

# 2020년 데이터
fomc_2020 = [
    {"month": "January", "days": "28-29", "sep": False},
    {"month": "March", "days": "3", "sep": False, "note": "unscheduled"},
    {"month": "March", "days": "15", "sep": False, "note": "unscheduled"},
    {"month": "March", "days": "17-18", "sep": True, "note": "cancelled"},
    {"month": "March", "days": "19", "sep": False, "note": "notation vote"},
    {"month": "March", "days": "23", "sep": False, "note": "notation vote"},
    {"month": "March", "days": "31", "sep": False, "note": "notation vote"},
    {"month": "April", "days": "28-29", "sep": False},
    {"month": "June", "days": "9-10", "sep": True},
    {"month": "July", "days": "28-29", "sep": False},
    {"month": "August", "days": "27", "sep": False, "note": "notation vote"},
    {"month": "September", "days": "15-16", "sep": True},
    {"month": "November", "days": "4-5", "sep": False},
    {"month": "December", "days": "15-16", "sep": True}
]

# 2021년 데이터
fomc_2021 = [
    {"month": "January", "days": "26-27", "sep": False},
    {"month": "March", "days": "16-17", "sep": True},
    {"month": "April", "days": "27-28", "sep": False},
    {"month": "June", "days": "15-16", "sep": True},
    {"month": "July", "days": "27-28", "sep": False},
    {"month": "September", "days": "21-22", "sep": True},
    {"month": "November", "days": "2-3", "sep": False},
    {"month": "December", "days": "14-15", "sep": True}
]

# 2022년 데이터 
fomc_2022 = [
    {"month": "January", "days": "25-26", "sep": False},
    {"month": "March", "days": "15-16", "sep": True},
    {"month": "May", "days": "3-4", "sep": False},
    {"month": "June", "days": "14-15", "sep": True},
    {"month": "July", "days": "26-27", "sep": False},
    {"month": "September", "days": "20-21", "sep": True},
    {"month": "November", "days": "1-2", "sep": False},
    {"month": "December", "days": "13-14", "sep": True}
]

# 비슷한 방식으로 2023년, 2024년 데이터도 추가 가능

# 모든 연도 데이터 합치기
for meetings, year in [(fomc_2020, 2020), (fomc_2021, 2021), (fomc_2022, 2022)]:
    for meeting in meetings:
        meeting_data = {
            "year": year,
            "month": meeting["month"],
            "days": meeting["days"],
            "sep": meeting["sep"],  # Summary of Economic Projections 포함 여부
            "note": meeting.get("note", "")
        }
        fomc_data.append(meeting_data)

# 데이터프레임 생성
fomc_df = pd.DataFrame(fomc_data)

# 날짜 처리
month_to_num = {
    "January": 1, "February": 2, "March": 3, "April": 4, "May": 5, "June": 6,
    "July": 7, "August": 8, "September": 9, "October": 10, "November": 11, "December": 12,
    "Jan/Feb": 1, "Oct/Nov": 10  # 특수 케이스
}

fomc_df["month_num"] = fomc_df["month"].map(month_to_num)

# 날짜 추출 및 변환 함수
def create_date(row):
    # 날짜 형식 처리 (예: "28-29" -> "28")
    day = row["days"].split("-")[0].split(" ")[0]  # 첫 번째 날만 사용
    try:
        return pd.Timestamp(f"{row['year']}-{row['month_num']}-{int(day)}")
    except:
        return pd.NaT  # 날짜 변환이 어려운 경우

fomc_df["fomc_date"] = fomc_df.apply(create_date, axis=1)

# 추가 처리: 날짜 정보가 없는 경우 등 결측치 처리
fomc_df = fomc_df.dropna(subset=["fomc_date"])

print(fomc_df[["year", "month", "days", "fomc_date", "sep", "note"]].head())

   year    month   days  fomc_date    sep           note
0  2020  January  28-29 2020-01-28  False               
1  2020    March      3 2020-03-03  False    unscheduled
2  2020    March     15 2020-03-15  False    unscheduled
3  2020    March  17-18 2020-03-17   True      cancelled
4  2020    March     19 2020-03-19  False  notation vote


In [9]:
# FOMC 발표일 리스트 (이전 단계에서 정의된 fomc_df)
fomc_df["fomc_date"] = pd.to_datetime(fomc_df["fomc_date"])

# S&P500 가격 데이터 (yfinance에서 가져온 데이터)
sp500["Date"] = pd.to_datetime(sp500["Date"])
sp500.set_index("Date", inplace=True)

# 수익률 컬럼 추가 (종가 기준)
sp500["daily_return"] = sp500["Close"].pct_change()

# 분석용 윈도우 설정 (±5영업일)
window = 5

# 이벤트 윈도우 데이터를 저장할 리스트
event_windows = []

# 각 FOMC 날짜에 대해 윈도우 수익률 계산
for event_date in fomc_df["fomc_date"]:
    # event_date 기준 ±5일의 날짜 인덱스 추출
    try:
        # 기준일 포함 ±5일치 수익률 추출
        event_window = sp500.loc[event_date - pd.Timedelta(days=10): event_date + pd.Timedelta(days=10)]
        event_window = event_window.reset_index()

        # FOMC 발표일 기준으로 몇일 전/후인지 계산
        event_window["day_offset"] = (event_window["Date"] - event_date).dt.days

        # 수익률 누적합 계산 (수익률 자체도 사용 가능)
        event_window["cumulative_return"] = (1 + event_window["daily_return"]).cumprod() - 1

        # 이벤트 + 해당 날짜 정보 저장
        event_window["fomc_date"] = event_date

        event_windows.append(event_window)

    except Exception as e:
        print(f"날짜 {event_date} 처리 중 오류 발생: {e}")

# 모든 윈도우 데이터를 하나의 DataFrame으로 합치기
event_all_df = pd.concat(event_windows)
event_all_df.head()


Price,Date,Close,High,Low,Open,Volume,daily_return,day_offset,cumulative_return,fomc_date
Ticker,,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC,,,,
0,2020-01-21,3320.790039,3329.790039,3316.610107,3321.030029,4104840000,-0.002652,-7,-0.002652,2020-01-28
1,2020-01-22,3321.750000,3337.770020,3320.040039,3330.020020,3623780000,0.000289,-6,-0.002364,2020-01-28
2,2020-01-23,3325.540039,3326.879883,3301.870117,3315.770020,3766710000,0.001141,-5,-0.001225,2020-01-28
3,2020-01-24,3295.469971,3333.179932,3281.530029,3333.100098,3708780000,-0.009042,-4,-0.010256,2020-01-28
4,2020-01-27,3243.629883,3258.850098,3234.500000,3247.159912,3831050000,-0.015731,-1,-0.025826,2020-01-28
